# Mujoco Control 
This tutorial introduces how to set up closed-loop control system for multi-body robotic systems in Mujoco. 


In [3]:
import mujoco
import mediapy as media
import numpy as np

import mujoco.viewer as viewer
modelxml = """
<mujoco model="CartPole">
  <compiler eulerseq="XYZ"/>
  <default>
    <default class="unused"/>
  </default>
  <asset>
    <texture name="grid" type="2d" builtin="checker" rgb1="0.1 0.2 0.3" rgb2="0.2 0.3 0.4" width="512" height="512"/>
    <material name="grid" class="unused" texture="grid" texrepeat="1 1" texuniform="true" reflectance="0.2"/>
  </asset>
  <worldbody>
    <geom name="floor" class="unused" type="plane" condim="3" size="0 0 0.05" material="grid" pos="0 0 -1"/>
    <body name="Cart" pos="0 0 0" euler="0 -0 0">
    <!-- change inertia, different from sdf -->
    <!-- For this model case, with the cart not having any rotational
             degrees of freedom, the values of the inertia matrix do not
             participate in the model. Therefore we just set them to zero
             (or near to zero since sdformat does not allow exact zeroes
             for inertia values). -->
      <inertial pos="0 0 0" mass="5" diaginertia="0.00000000001 0.00000000001 0.00000000001"/>
      <geom name="cart_visual" class="unused" type="box" contype="0" conaffinity="0" group="0" size="0.12 0.06 0.06" pos="0 0 0" euler="0 -0 0"/>
      <joint name="CartSlider" class="unused" type="slide" pos="0 0 0" axis="1 0 0"/>
      <body name="Pole" pos="0 0 -0.5" euler="0 -0 0">
        <inertial pos="0 0 0" mass="1" diaginertia="0.00000000001 0.00000000001 0.00000000001"/>
        <geom name="pole_point_mass" class="unused" type="sphere" contype="0" conaffinity="0" group="0" size="0.05" pos="0 0 0" euler="0 -0 0"/>
        <geom name="pole_rod" class="unused" type="cylinder" contype="0" conaffinity="0" group="0" size="0.025 0.25" pos="0 0 0.25" euler="0 -0 0"/>
        <joint name="PolePin" class="unused" type="hinge" pos="0 0 0.5" axis="0 -1 0"/>
      </body>
    </body>
  </worldbody>
  <sensor>
        <!-- joint position sensing -->
        <jointpos joint="CartSlider" name="cart_p"/>
        <jointpos joint="PolePin" name="pole_theta"/>
        <!-- joint velocity sensing -->
        <jointvel joint="CartSlider" name="cart_v"/>
        <jointvel joint="PolePin" name="pole_w"/>
  </sensor>
  <actuator>
    <motor joint="CartSlider"/>
  </actuator>
  <keyframe>
    <key name="off1" qpos="0 3.15"/>
  </keyframe>
</mujoco>
"""
model = mujoco.MjModel.from_xml_string(modelxml)
data = mujoco.MjData(model)
mujoco.mj_resetDataKeyframe(model, data, 0)
viewer.launch(model,data)

### Simulatioin of open-loop resposne 
now let's simulate the physics


In [ ]:
duration = 3    # (seconds)
framerate = 30  # (Hz)

# Simulate and display video.
frames = []
mujoco.mj_resetDataKeyframe(model, data, 0)  # Reset the state to keyframe 0
with mujoco.Renderer(model) as renderer:
  while data.time < duration:
    mujoco.mj_step(model, data)
    if len(frames) < data.time * framerate:
      renderer.update_scene(data)
      pixels = renderer.render()
      frames.append(pixels)
media.show_video(frames, fps=framerate, codec='gif')

In [5]:
# define customized controller which returns the feedback control action
# if we want to use control callback, we need to set data.ctrl, the returned value does not matter in this case
def myControl(model, data):
    x = np.hstack((data.qpos,data.qvel))
    xref = np.array([0, np.pi, 0, 0])
    x_error = x-xref
    K = np.array([-0.5, 0.5, 0, 0])
    u = K@x_error
    data.ctrl = u
    return u
    

In [17]:
import mujoco.viewer as viewer
import time

mujoco.mj_resetDataKeyframe(model, data, 0)  # Reset the state to keyframe 0

# 
with viewer.launch_passive(model, data) as viewer:  
  # launch_passive means all the simulation should be done by the user 
  
  start = time.time()
  while viewer.is_running() and time.time() - start < 10:
    step_start = time.time()
    data.ctrl = 0.2*myControl(model,data)
    mujoco.mj_step(model, data)

  # let viewer show updated info
    viewer.sync()
    
  # #  make sure the while loop is called every sampling period 
    # computation inside the loop may take some nontrivial time. 
    time_until_next_step = model.opt.timestep - (time.time() - step_start)
    if time_until_next_step > 0:
      time.sleep(time_until_next_step)

### Simulation method 1: 
interactive simulation, the viewer can respond to user's input during the simulation

In [6]:
# simulating the closed-loop system by using the control callback

import mujoco.viewer as viewer
mujoco.mj_resetDataKeyframe(model, data, 0)  # Reset the state to keyframe 0
mujoco.set_mjcb_control(myControl)
viewer.launch(model,data)

### Simuation method 2:
Passive simulation, the user has the control over stepping the physics 

In [8]:
import mujoco.viewer as viewer
import time

mujoco.mj_resetDataKeyframe(model, data, 0)  # Reset the state to keyframe 0
with viewer.launch_passive(model,data) as viewer:
    while viewer.is_running() and data.time < 5:
        data.ctrl = 20*myControl(model,data)  # apply control
        mujoco.mj_step(model, data)            # step xdot= f(x,u)
        viewer.sync()
        time.sleep(model.opt.timestep)        
